In [1]:
gpu=7
CUDA_VISIBLE_DEVICES=gpu

In [2]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
import time
import copy
from random import shuffle

import tqdm.notebook as tqdm

import sklearn
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from PIL import Image
import cv2

# import osa
import shutil
import seaborn as sns
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision
from glob import glob
from sklearn.metrics import roc_curve,auc, precision_score,precision_recall_curve,recall_score,precision_recall_fscore_support,confusion_matrix
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import models
from prettytable import PrettyTable
print(torch.cuda.is_available())
import warnings
warnings.filterwarnings("ignore")
# import pandas as pd
print(torch.cuda.get_device_properties(0).total_memory)
print(torch.cuda.memory_allocated())
gpu_id = gpu

True
11721506816
0


In [3]:
import torch
torch.__version__


'1.10.1+cu102'

In [4]:
# pip install ray[tune]

In [5]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [6]:
train_df=pd.read_csv("/DATA/chowdari1/DATA/csv/ppr_nih_train.csv")
val_df=pd.read_csv("/DATA/chowdari1/DATA/csv/ppr_nih_val.csv")
test_df=pd.read_csv("/DATA/chowdari1/DATA/csv/ppr_nih_test.csv")
train_df=train_df.sample(frac=1,random_state=172)
val_df=val_df.sample(frac=1,random_state=172)
test_df=test_df.sample(frac=1,random_state=172)

In [7]:
train_df.head()

,Unnamed: 0,Image index,target,path
6421,6421,00019823_001.png,1,/DATA/chowdari1/DATA/dataset/nih/images_009/im...
5833,5833,00018194_000.png,0,/DATA/chowdari1/DATA/dataset/nih/images_008/im...
4382,4382,00014840_000.png,0,/DATA/chowdari1/DATA/dataset/nih/images_007/im...
1977,1977,00007292_003.png,1,/DATA/chowdari1/DATA/dataset/nih/images_004/im...
2409,2409,00009191_000.png,1,/DATA/chowdari1/DATA/dataset/nih/images_004/im...


In [8]:

# for i in range(df.shape[0]):
#     df['path'][i]+='.png'
from torch.utils.data import Dataset, DataLoader

class CustomImageDataset(Dataset):
    def __init__(self, df, img_dir, transform,start,count):
        self.ipaths = df["path"][start:count].to_numpy()
        self.target=df["target"][start:count].to_numpy()
        self.transform = transform
        self.count=count
        self.img_dir=img_dir


    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        img_path=self.img_dir+self.ipaths[idx]
        image = Image.open(img_path).convert('RGB')
        label=self.target[idx]
        image = self.transform(image)
        return image, label

In [9]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.449,),(0.226,))
])

In [10]:
train_df.shape, val_df.shape

((8574, 4), (1706, 4))

In [11]:
def load_data(data_dir=''):
    train_data=CustomImageDataset(train_df,'',transform,0,512)
    valid_data=CustomImageDataset(val_df,'',transform,0,256)

    return train_data, valid_data

In [12]:
# 8448,1536
# train_data=CustomImageDataset(train_df,'',transform,0,512)
# valid_data=CustomImageDataset(val_df,'',transform,0,256)

In [13]:
# train_data=CustomImageDataset(train_df,'',transform,0,8448)
# valid_data=CustomImageDataset(val_df,'',transform,0,1536)
# # test_data=CustomImageDataset(df,'/DATA/dataset/vinbig/vinbig/trainpng/',transform,7000+6000,df.shape[0])
# print(len(train_data))
# # print(len(test_data))
# trainloader=torch.utils.data.DataLoader(train_data, batch_size=256)
# valloader=torch.utils.data.DataLoader(valid_data, batch_size=256)
# # testloader=torch.utils.data.DataLoader(test_data, batch_size=256)
# print(trainloader)
# # print(len(testloader))
# # dataloaders = {"train":trainloader, "val":valloader, "test": testloader}
# dataloaders = {"train":trainloader, "val":valloader}
# # data_sizes = {x: len(dataloaders[x].sampler) for x in ['train','val','test']}
# data_sizes = {x: len(dataloaders[x].sampler) for x in ['train','val']}

# data_sizes

In [14]:
import torch
torch.cuda.empty_cache()
device=torch.device("cuda:7")
print("Training on GPU... Ready for HyperJump...")

Training on GPU... Ready for HyperJump...


In [15]:
torch.cuda.set_device(gpu_id)

In [16]:
class VariationalEncoder(nn.Module):
    def __init__(self,l1=224,l2=4):
        super(VariationalEncoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, stride=2, padding=1)
        # out_width = (28+2-5)/2+1 = 27/2+1 = 13
        self.conv2 = nn.Conv2d(8,16, 3, stride=2, padding=1)
        self.batch2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, 3, stride=2, padding=0)
        # out_width = (14-5)/2+1 = 5
        #self.drop1=nn.Dropout2d(p=0.3) 
        # 6 * 6 * 16 = 576
        latent_dims=l2
        self.linear1 = nn.Linear(23328, l1)
        self.linear2 = nn.Linear(l1, latent_dims)
        self.linear3 = nn.Linear(l1, latent_dims)

        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.cuda() # hack to get sampling on the GPU
        self.N.scale = self.N.scale.cuda()
        self.kl = 0

    def forward(self, x):
        #print(x.shape)
        global en
        x = x.to(device)
        x = F.relu(self.conv1(x))
        x = F.relu(self.batch2(self.conv2(x)))
        x = F.relu(self.conv3(x))
        #print(x.shape)
        x = torch.flatten(x, start_dim=1)
        #print(x.shape)
        x = F.relu(self.linear1(x))
        mu =  self.linear2(x)
        sigma = torch.exp(self.linear3(x))
        z = mu + sigma*self.N.sample(mu.shape)
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        # print(en,'hello',z[0])
        # en+=1
        return z

In [17]:
class Decoder(nn.Module):
    
    def __init__(self,l1=224,l2=4 ):
        super().__init__()

        ### Linear section
        latent_dims=l2
        self.decoder_lin = nn.Sequential(
            # First linear layer
            nn.Linear(latent_dims, l1),
            nn.ReLU(True),
            # Second linear layer
            nn.Linear(l1, 23328),
            nn.ReLU(True)
        )

        ### Unflatten
        self.unflatten = nn.Unflatten(dim=1, unflattened_size=(32, 27, 27))

        ### Convolutional section
        self.decoder_conv = nn.Sequential(
            # First transposed convolution
            nn.ConvTranspose2d(32, 16, 3, stride=2, output_padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            # Second transposed convolution
            nn.ConvTranspose2d(16, 8, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(True),
            # Third transposed convolution
            nn.ConvTranspose2d(8, 1, 3, stride=2, padding=1, output_padding=1)
        )
        
    def forward(self, x):
        global de
        # Apply linear layers
        x = self.decoder_lin(x)
        # print('de1',x)
        # Unflatten
        x = self.unflatten(x)
        # print('de2',x)
        # Apply transposed convolutions
        x = self.decoder_conv(x)
        # print('de3',x)
        # Apply a sigmoid to force the output to be between 0 and 1 (valid pixel values)
        x = torch.sigmoid(x)
        # print(de,'de4',x[0][0][0][0])
        # de+=1
        return x

In [18]:
# class VariationalAutoencoder(nn.Module):
#     def __init__(self,l1=224,l2=4):
#         super(VariationalAutoencoder, self).__init__()
#         #  latent_dims
#         # latent_dims=int(l2)
#         self.encoder = VariationalEncoder(l1,l2)
#         self.decoder = Decoder(l1,l2)

#     def forward(self, x):
#         # x = x.to(device)
#         z = self.encoder(x)
#         # print(z)
#         return self.decoder(z)

In [47]:
class VariationalAutoencoder(nn.Module,):
    def __init__(self,l1=224,l2=4):
        super(VariationalAutoencoder, self).__init__()
        #  latent_dims
        # latent_dims=int(l2)
        # self.encoder = VariationalEncoder(l1,l2)
        # self.decoder = Decoder(l1,l2)
        self.l1=l1
        self.l2=l2
        # self.a=l1+l2
        self.kl=0

    def forward(self, x):
        # x = x.to(device)
        encoder = VariationalEncoder(self.l1,self.l2)
        decoder = Decoder(self.l1,self.l2)
        z=encoder(x)
        self.kl=encoder.kl
        return decoder(x)
        # z = self.encoder(x)
        # z=5+x.shape[0]
        # z=self.a+1
        # # print(z)
        # # return self.decoder(z)
        # return z

In [20]:
torch.cuda.empty_cache()
device=torch.device("cuda:7")

In [21]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [48]:
vae=VariationalAutoencoder(config['l1'],config['l2'])
vae.parameters()

<generator object Module.parameters at 0x7f6da4679048>

In [58]:
import datetime
def train_mana(config):
    vae=VariationalAutoencoder(config['l1'],config['l2'])
    # net=Net(config['l1'],config['l2'])
    # torch.cuda.empty_cache()
    # device=torch.device("cuda:7")
    # vae.to(device)
    optimizer = torch.optim.Adam(vae.parameters(), lr=config['lr'], weight_decay=1e-5)
    trainset, testset= load_data()
    trainloader=torch.utils.data.DataLoader(trainset, 
        batch_size=int(config["batch_size"]))
    valloader=torch.utils.data.DataLoader(testset, 
        batch_size=int(config["batch_size"]))
    
    # for epoch in range(5):
    # num_epochs=2
    val_loss=0
    # for epoch in tqdm.tqdm(range(num_epochs)):
    # for epoch in range(num_epochs):
        # # print('Epoch {}/{}'.format( epoch+1, num_epochs))       
        # # print (datetime. datetime. now())
        # # print('-' * 10)
        # epoch_steps=0
        # train_loss = 0.0
        # # for x, _ in enumerate(trainloader,0):
        # for x,_ in trainloader:
        #     x=x.to(device)
        #     x_hat=vae(x)
        #     loss= ((x - x_hat)**2).sum() + vae.encoder.kl  

        #     optimizer.zero_grad()
        #     loss.backward()
        #     optimizer.step()
        #     train_loss=+loss.item()

        #     epoch_steps+=1
        # val_loss=0
        # val_steps=0
        # # for x,_ in enumerate(valloader,0):
        # for x,_ in valloader:
        #     with torch.no_grad():
        #         x = x.to(device)
        #         encoded_data = vae.encoder(x)
        #         x_hat = vae(x)
        #         loss = ((x - x_hat)**2).sum() + vae.encoder.kl
        #         # val_loss += loss.item()
        #         val_loss += loss.cpu().numpy()
        #         val_steps+=1
        # val_loss=val_loss / len(valloader.dataset)
        # print(val_loss)
        
    y=intermediate(val_loss,config['l1'],config['l2'],config['batch_size'],config['lr'])
    tune.report(score=y)
    print("finished training ---------")


In [50]:
def intermediate(a,b,c,d,e):
    score=a+b+c+d+e
    return score

In [51]:
config={
    'l1': 112,
    'l2': 20,
    'lr': 1e-3,
    "batch_size": 256
}

In [53]:
train_mana(config)

finished training ---------


In [54]:
config={
    'l1': tune.grid_search([112,224]),
    'l2': tune.grid_search([4,20]),
    'lr':tune.choice([1e-3,1e-4]),
    "batch_size" : tune.choice([256,256])
}

In [33]:
config

{'l1': {'grid_search': [112, 224]},
 'l2': {'grid_search': [4, 20]},
 'lr': <ray.tune.sample.Categorical at 0x7f3c407ace48>,
 'batch_size': <ray.tune.sample.Categorical at 0x7f3c407ac128>}

In [56]:
s

{'l1': 112, 'l2': 4, 'lr': 0.001, 'batch_size': 256}

In [59]:
print('start-----')
analysis = tune.run(
    train_mana,
    config=config
)
s=analysis.get_best_config(metric="score", mode="min")
print("best config: ", analysis.get_best_config(metric="score", mode="max"))
# df = analysis.dataframe(metric="score", mode="min")
# all_dataframes = analysis.trial_dataframes
print('heloo------------------')

2022-02-09 13:01:17,660	WARNING tune.py:580 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


start-----


Trial name,status,loc,batch_size,l1,l2,lr
train_mana_44e0b_00000,RUNNING,172.25.0.208:44087,256,112,4,0.001
train_mana_44e0b_00001,PENDING,,256,224,4,0.001
train_mana_44e0b_00002,PENDING,,256,112,20,0.001
train_mana_44e0b_00003,PENDING,,256,224,20,0.0001


(train_mana pid=44087) 2022-02-09 13:01:20,234	ERROR function_runner.py:268 -- Runner Thread raised error.
(train_mana pid=44087) Traceback (most recent call last):
(train_mana pid=44087)   File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 262, in run
(train_mana pid=44087)     self._entrypoint()
(train_mana pid=44087)   File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 331, in entrypoint
(train_mana pid=44087)     self._status_reporter.get_checkpoint())
(train_mana pid=44087)   File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/util/tracing/tracing_helper.py", line 451, in _resume_span
(train_mana pid=44087)     return method(self, *_args, **_kwargs)
(train_mana pid=44087)   File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 600, in _trainable_func
(train_mana pid=44087)     output = fn()
(train_mana pid=44087)   File "/tmp/ipykernel_30425/2091847976.p

Result for train_mana_44e0b_00000:
  date: 2022-02-09_13-01-20
  experiment_id: 73ace2e347a44308b8c2928e20a0da3d
  hostname: BOSS8-OS
  node_ip: 172.25.0.208
  pid: 44087
  timestamp: 1644391880
  trial_id: 44e0b_00000
  


(train_mana pid=44275) 2022-02-09 13:01:22,245	ERROR function_runner.py:268 -- Runner Thread raised error.
(train_mana pid=44275) Traceback (most recent call last):
(train_mana pid=44275)   File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 262, in run
(train_mana pid=44275)     self._entrypoint()
(train_mana pid=44275)   File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 331, in entrypoint
(train_mana pid=44275)     self._status_reporter.get_checkpoint())
(train_mana pid=44275)   File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/util/tracing/tracing_helper.py", line 451, in _resume_span
(train_mana pid=44275)     return method(self, *_args, **_kwargs)
(train_mana pid=44275)   File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 600, in _trainable_func
(train_mana pid=44275)     output = fn()
(train_mana pid=44275)   File "/tmp/ipykernel_30425/2091847976.p

Result for train_mana_44e0b_00002:
  date: 2022-02-09_13-01-22
  experiment_id: c2a56b41fd13464684b44a70c04ac068
  hostname: BOSS8-OS
  node_ip: 172.25.0.208
  pid: 44260
  timestamp: 1644391882
  trial_id: 44e0b_00002
  
Result for train_mana_44e0b_00001:
  date: 2022-02-09_13-01-22
  experiment_id: 7d9aaa88921f494dacbe7036449207f6
  hostname: BOSS8-OS
  node_ip: 172.25.0.208
  pid: 44275
  timestamp: 1644391882
  trial_id: 44e0b_00001
  


2022-02-09 13:01:22,603	ERROR trial_runner.py:927 -- Trial train_mana_44e0b_00003: Error processing event.
Traceback (most recent call last):
  File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 893, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 707, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/worker.py", line 1733, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(TuneError): ray::ImplicitFunc.train() (pid=44239, ip=172.25.0.208, repr=train_mana)
  File "/DATA/chowdari1/.local/lib/python3.7/site-packages/ray/tune/trainable.py", line 315, in train
    result =

Result for train_mana_44e0b_00003:
  date: 2022-02-09_13-01-22
  experiment_id: 632980fea8a44e5391a6b948c12a9e72
  hostname: BOSS8-OS
  node_ip: 172.25.0.208
  pid: 44239
  timestamp: 1644391882
  trial_id: 44e0b_00003
  


TuneError: ('Trials did not complete', [train_mana_44e0b_00000, train_mana_44e0b_00001, train_mana_44e0b_00002, train_mana_44e0b_00003])

In [30]:
import datetime

def train(config,checkpoint_dir=None,data_dir=None):
    vae=VariationalAutoencoder(config['l1'],config['l2'])

    torch.cuda.empty_cache()
    device=torch.device("cuda:7")

    vae.to(device)
    optimizer = torch.optim.Adam(vae.parameters(), lr=config['lr'], weight_decay=1e-5)

    if checkpoint_dir:
        model_state, optimizer_state=torch.load(
            os.path.join(checkpoint_dir,"checkpoint"))
        vae.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset= load_data(data_dir)
    trainloader=torch.utils.data.DataLoader(trainset, 
        batch_size=int(config["batch_size"]), num_workers=8)
    valloader=torch.utils.data.DataLoader(testset, 
        batch_size=int(config["batch_size"]), num_workers=8)

    for epoch in range(10):
        print('Epoch {}/{}'.format( epoch+1, num_epochs))       
        print (datetime. datetime. now())
        print('-' * 10)
        epoch_steps=0
        train_loss = 0.0
        for x, _ in enumerate(trainloader,0):
            x=x.to(device)
            x_hat=vae(x)
            loss= ((x - x_hat)**2).sum() + vae.encoder.kl  

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss=+loss.item()

            epoch_steps+=1
        val_loss=0
        val_steps=0
        for x,_ in enumerate(valloader,0):
            with torch.no_grad():
                x = x.to(device)
                encoded_data = vae.encoder(x)
                x_hat = vae(x)
                loss = ((x - x_hat)**2).sum() + vae.encoder.kl
                # val_loss += loss.item()
                val_loss += loss.cpu().numpy()
                val_steps+=1
        
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path=os.path.join(checkpoint_dir, "checkpoint")
            torch.save((vae.state_dict(), optimizer.state_dict()),path)
        
        tune.report(loss=(train_loss / len(trainloader.dataset)), accuracy=(val_loss / len(valloader.dataset)))
    print("finished training")
                
                


In [31]:
def test_accuracy(vae, device=device):
    trainset, testset=load_data()
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)
    
    val_loss=0
    with torch.no_grad():
        for x,_ in enumerate(testloader,0):
            with torch.no_grad():
                x = x.to(device)
                encoded_data = vae.encoder(x)
                x_hat = vae(x)
                loss = ((x - x_hat)**2).sum() + vae.encoder.kl
                # val_loss += loss.item()
                val_loss += loss.cpu().numpy()
                # val_steps+=1
    return  (val_loss / len(testloader.dataset))  

In [ ]:
# from functools import partial
# config={
#         "l1": 224,
#         "l2": tune.choice([4,20]),
#         "lr": tune.loguniform(1e-4, 1e-1),
#         "lr": tune.choice([1e-4, 1e-1]),
#         "batch_size": 256
#     }
# result = tune.run(
#         partial(train, data_dir=''),     
#         config=config,
#         # resources_per_trail={'gpu': 1},
#         # resources_per_trial={"cpu": 2, "gpu": gpus_per_trail}
#         )
# print("Best config: ", result.get_best_config(metric="mean_accuracy"))

In [32]:
from functools import partial
def main(num_samples=10, max_num_epochs=10, gpus_per_trail=2):
    data_dir=''
    load_data('')
    config={
        "l1": tune.choice([224]),
        "l2": tune.choice([4,20]),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([256])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trail},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)
    
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))
    
    best_trained_model = VariationalAutoencoder(best_trial.config["l1"], best_trial.config["l2"])
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


In [ ]:
if __name__ == "__main__":
    main(num_samples=10, max_num_epochs=10, gpus_per_trail=0)

In [21]:
def train_epoch(vae, device, dataloader, optimizer):
    # Set train mode for both the encoder and the decoder
    vae.train()
    train_loss = 0.0
    phase='train'
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    # for x, _ in dataloader: 
    for x, _ in tqdm.tqdm(dataloader, desc=phase, leave=False):
        # Move tensor to the proper device
        x = x.to(device)
        x_hat = vae(x)
        # Evaluate loss
        loss = ((x - x_hat)**2).sum() + vae.encoder.kl

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print batch loss
        # print('\t partial train loss (single batch): %f' % (loss.item()))
        train_loss+=loss.item()

    return train_loss / len(dataloader.dataset), vae

In [23]:
def test_epoch(vae, device, dataloader):
    # Set evaluation mode for encoder and decoder
    vae.eval()
    val_loss = 0.0
    phase='test'
    with torch.no_grad(): # No need to track the gradients
        for x, _ in tqdm.tqdm(dataloader, desc=phase, leave=False):
            # Move tensor to the proper device
            x = x.to(device)
            # Encode data
            encoded_data = vae.encoder(x)
            # Decode data
            x_hat = vae(x)
            loss = ((x - x_hat)**2).sum() + vae.encoder.kl
            val_loss += loss.item()

    return val_loss / len(dataloader.dataset)

In [24]:
# 1280
test_data=CustomImageDataset(test_df,'',transform,0,256)
testloader=torch.utils.data.DataLoader(test_data, batch_size=256)
len(testloader.sampler)

256

In [25]:
# test_data=CustomImageDataset(test_df,'',transform,0,1280)
# testloader=torch.utils.data.DataLoader(test_data, batch_size=256)
# len(testloader.sampler)
def test_loss(vae, device, dataloader):
    # Set evaluation mode for encoder and decoder
    vae.eval()
    val_loss = 0.0
    phase='test'
    with torch.no_grad(): # No need to track the gradients
        for x, _ in tqdm.tqdm(dataloader, desc=phase, leave=False):
            # Move tensor to the proper device
            x = x.to(device)
            # Encode data
            encoded_data = vae.encoder(x)
            # Decode data
            x_hat = vae(x)
            loss = ((x - x_hat)**2).sum() + vae.encoder.kl
            val_loss += loss.item()

    return val_loss / len(dataloader.dataset)

In [26]:
test_dataset=CustomImageDataset(test_df,'',transform,0,256)
def plot_ae_outputs(encoder,decoder,n=5):
    plt.figure(figsize=(10,4.5))
    for i in range(n):
      ax = plt.subplot(2,n,i+1)
      img = test_dataset[i][0].unsqueeze(0).to(device)
      encoder.eval()
      decoder.eval()
      with torch.no_grad():
         rec_img  = decoder(encoder(img))
      plt.imshow(img.cpu().squeeze().numpy(), cmap='gist_gray')
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)  
      if i == n//2:
        ax.set_title('Original images')
      ax = plt.subplot(2, n, i + 1 + n)
      plt.imshow(rec_img.cpu().squeeze().numpy(), cmap='gist_gray')  
      plt.imshow(rec_img.cpu().squeeze().numpy(), cmap='gist_gray')  
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)  
      if i == n//2:
         ax.set_title('Reconstructed images')
    plt.show() 

In [35]:
import datetime
num_epochs=3
dims=4
learn_rt=1e-3
def train(config,checkpoint_dir=None, data_dir=None):
    # vae=VariationalAutoencoder(latent_dims=config['latent_dim'])
    vae=VariationalAutoencoder(latent_dims=dims)
    # optim = torch.optim.Adam(vae.parameters(), lr=config['lr'], weight_decay=1e-5)
    optim = torch.optim.Adam(vae.parameters(), lr=learn_rt, weight_decay=1e-5)
    if checkpoint_dir:
        model_state, optimizer_state= torch.load(
            os.path.join(checkpoint_dir,"checkpoint"))
        vae.load_state_dict(model_state)
        optim.load_state_dict(optimizer_state)

    trainloader=torch.utils.data.DataLoader(train_data, batch_size=256)
    valloader=torch.utils.data.DataLoader(valid_data, batch_size=256)
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format( epoch+1, num_epochs))       
        print (datetime. datetime. now())
        print('-' * 10)
        train_loss,vae = train_epoch(vae,device,trainloader,optim)
        val_loss = test_epoch(vae,device,valloader)

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path=os.path.join(checkpoint_dir,'checkpoint')
            torch.save((vae.state_dict(),optim.state_dict(),path))
        
        tune.report(loss=val_loss, accuracy= train_loss)
    print("training over")


In [28]:
path= os.path.join('/DATA/chowdari1/vae', "checkpoint")
path

'/DATA/chowdari1/vae/checkpoint'

In [46]:
from functools import partial
# max_num_epoch=100

def main(max_num_epochs):
        print('num_epochs: ',max_num_epochs)
        config={
        "latent_dim": tune.choice([2,4,8,20]),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([256,64])
        }
        scheduler = ASHAScheduler(
                metric="loss",
                mode="min",
                max_t=max_num_epochs,
                grace_period=1,
                reduction_factor=2)
        # reporter = CLIReporter()
                # parameter_columns=["l1", "l2", "lr", "batch_size"],
                # metric_columns=["loss", "accuracy","training_iteration"])
        # num_epochs,config,checkpoint_dir=None, data_dir=None
        result = tune.run(
        partial(train ), 
         resources_per_trial={"cpu": 2, "gpu": 0},       
        config=config,
        scheduler=scheduler,
        # progress_reporter=reporter
        )

        best_trial = result.get_best_trial("loss", "min", "last")
        print("Best trial config: {}".format(best_trial.config))
        print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
        print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

        best_trained_model = VariationalAutoencoder(latent_dims=best_trail.config['latent_dim'])

        best_checkpoint_dir = best_trial.checkpoint.value
        model_state, optimizer_state = torch.load(os.path.join(
                best_checkpoint_dir, "checkpoint"))
        best_trained_model.load_state_dict(model_state)

        test_acc = test_loss(best_trained_model, device,testloader)
        print("Best trial test set accuracy: {}".format(test_acc))

# if __name__ =='main':
#         print('noooo')
#         main(5)
#         print('yes')

In [ ]:
if __name__ =='__main__':
    main(3)
    print('yes')

In [ ]:
if __name__ =='__main__':
    main(3)
    print('yes')

In [49]:
# num_epochs = 10
import datetime
def train(vae,num_epochs):
    best_test_loss = float('inf')
    best_model_wts = copy.deepcopy(vae.state_dict())
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format( epoch+1, num_epochs))       
        print (datetime. datetime. now())
        print('-' * 10)
        train_loss = train_epoch(vae,device,trainloader,optim)
        val_loss = test_epoch(vae,device,valloader)
        if(best_test_loss > val_loss):
            best_test_loss=val_loss
            best_model_wts = copy.deepcopy(vae.state_dict())
        print('\n EPOCH {}/{} \t train loss {:.3f} \t val loss {:.3f}'.format(epoch + 1, num_epochs,train_loss,val_loss))
    print('best_test_loss: ',best_test_loss)
    plot_ae_outputs(vae.encoder,vae.decoder,n=5)

    vae.load_state_dict(best_model_wts)
    return vae

In [ ]:
va=train(vae,2)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):

   train_loss = train_epoch(vae,device,trainloader,optim)
   val_loss = test_epoch(vae,device,valloader)
   print('\n EPOCH {}/{} \t train loss {:.3f} \t val loss {:.3f}'.format(epoch + 1, num_epochs,train_loss,val_loss))

plot_ae_outputs(vae.encoder,vae.decoder,n=5)

In [25]:
path="/DATA/radhasyam1/saved_models/vae.pth"
torch.save(vae.state_dict(), path)

In [50]:
path='/DATA/chowdari1/saved_models/vae/vae_big_nihppr_100ep.pth'
# model=densenet_Model(pretrained=True)
model= VariationalAutoencoder(latent_dims=d)
x_model=torch.load(path,map_location='cpu')
# x_model=x_model.to(device)
model.load_state_dict(x_model)
model=model.to(device)
model.eval()
print(path)

/DATA/chowdari1/saved_models/vae/vae_big_nihppr_100ep.pth


In [52]:
test_loss = test_epoch(model,device,testloader)
test_loss

21651.47109375

In [ ]:
plot_ae_outputs(model.encoder,model.decoder,n=5)

In [48]:
test_loss = test_epoch(va,device,testloader)
test_loss

36669.517578125

In [46]:
test_df.shape

(1344, 4)

In [51]:
test_data=CustomImageDataset(test_df,'',transform,0,1280)
testloader=torch.utils.data.DataLoader(test_data, batch_size=256)
len(testloader.sampler)

1280